In [1]:
import csv


ids = []
g = []
p = []

with open('train.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        ids.append(row[0])
        
        gs = []
        for c in row[1].lower():
            if c.isalpha() or c == "'":
                gs.append(c)
        
        g.append(gs)
        ph = row[2].split()
        p.append(ph)
        
graphemes = g
phonemes = p

In [2]:
align_data = []
words = []

p_cls = set([])
g_cls = set([])

p_cls_cnt = {}
p_total_cnt = 0

def word_from_align(a):
    w = ""
    for chunk in a:
        if len(chunk) > 1:
            w += chunk[0] + chunk[-1]
        else:
            w += chunk
    return w

with open('data.m-mAlign.2-2.delX.1-best.conYX.align') as f:
    align_data = f.readlines()
    for i in range(len(align_data)):
        align_data[i] = align_data[i].split('\t')
        align_data[i][0] = align_data[i][0].split('|')
        align_data[i][1] = align_data[i][1].split('|')
        align_data[i][0].remove('')
        align_data[i][1].remove('\n')
        words.append(word_from_align(align_data[i][0]))
        
        for c in align_data[i][0]:
            g_cls.add(c)
        
        for c in align_data[i][1]:
            p_cls.add(c)

In [3]:
print (align_data[0], words[0])

p_cls_id = {}
g_cls_id = {}

p_cls_id['~'] = 0
i = 1
for pc in p_cls:
    p_cls_id[pc] = i
    i += 1
    
g_cls_id['~'] = 0
i = 1
for gc in g_cls:
    g_cls_id[gc] = i
    i += 1
    
print (len(p_cls_id), len(g_cls_id))

[['k:n', 'o', 'x', 'v', 'i', 'l:l', 'e'], ['N', 'AA', 'K:S', 'V', 'IH', 'L', '_']] knoxville
181 315


In [4]:
X = []
Y = []

def left_ctx(a, i, ctx_len=2):
    ctx = []
    j = i-1
    while j >= 0 and len(ctx) < ctx_len:
        if len(a[j]) > 1:
            if len(ctx) < ctx_len:
                ctx.append(a[j][-1])
            if len(ctx) < ctx_len:
                ctx.append(a[j][0])
        else:
            ctx.append(a[j])
        j -= 1
    
    while len(ctx) < ctx_len:
        ctx.append('~')
    
    return ctx[::-1]

def right_ctx(a, i, ctx_len=2):
    ctx = []
    j = i + 1
    while j < len(a) and len(ctx) < ctx_len:
        if len(a[j]) > 1:
            if len(ctx) < ctx_len:
                ctx.append(a[j][0])
            if len(ctx) < ctx_len:
                ctx.append(a[j][-1])
        else:
            ctx.append(a[j])
        j += 1
        
    while len(ctx) < ctx_len:
        ctx.append('~')
    
    return ctx

def get_xy(a, w):
    x = []
    y = []
    ri = 0
    for i in range(len(a)):
        if i == len(a) - 1 and len(a[i]) == 1:
            break
        l = []
        if len(a[i]) > 1:
            l = left_ctx(a, i) + [a[i][0], a[i][-1]] + right_ctx(a, i)
            
            ll = [c for c in l[1::]]
            if ri+4 < len(w):
                ll.append(w[ri+4])
            else:
                ll.append('~')
            ll = [ord(c) for c in ll]
            
            l = [ord(c) for c in l]
            x.append( l )
            y.append(1)
            x.append( ll )
            y.append(0)
        else:
            if len(a[i+1]) == 1:
                l = left_ctx(a, i) + [a[i], a[i+1]] + right_ctx(a, i+1)
            else:
                l = left_ctx(a, i) + [a[i], a[i+1][0], a[i+1][-1]] + right_ctx(a, i+1, 1)
            y.append(0)
            l = [ord(c) for c in l]
            x.append( l )
            
        ri += min(len(a[i]), 2)
        
    return (x, y)

for k in range(len(align_data)):
    xa, ya = get_xy(align_data[k][0], words[k])
    for x,y in zip(xa, ya):
        X.append(x)
        Y.append(y)

In [5]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X, Y)

In [6]:
def word_to_window(w, focus, l_ctx_len=2, r_ctx_len=2):
    lctx = []
    rctx = []
    j = focus - 1
    while j >= 0 and len(lctx) < l_ctx_len:
        lctx.append(w[j])
        j -= 1
    while len(lctx) < l_ctx_len:
        lctx.append('~')
    j = focus + 2
    while j < len(w) and len(rctx) < r_ctx_len:
        rctx.append(w[j])
        j += 1
    while len(rctx) < r_ctx_len:
        rctx.append('~')
    l = lctx[::-1] + [w[focus], w[focus+1]] + rctx
    l = [ord(c) for c in l]
    return l

def letter_chunking(w):
    f = []
    for focus in range(len(w)-1):
        window = word_to_window(w, focus)
        f.append(clf.predict([window]))
    f.append(0)
    for i in range(1, len(w)):
        if f[i] == 1 and f[i-1] == 1:
            f[i] = 0
    chunks = []
    i = 0
    while i < len(w):
        if f[i] == 1:
            chunks.append([w[i], w[i+1]])
            i += 2
        else:
            chunks.append([w[i]])
            i += 1
    return chunks

In [7]:
XX = []
YY = []

def get_xy2(g, p):
    x = []
    y = []
    for i in range(len(g)):
        l_ctx = []
        r_ctx = []
        j = i - 1
        while j >= 0 and len(l_ctx) < 4:
            l_ctx.append(g_cls_id[g[j]])
            l_ctx.append(p_cls_id[p[j]])
        while len(l_ctx) < 4:
            l_ctx.append(g_cls_id['~'])
            l_ctx.append(p_cls_id['_'])
            
        if i < len(g) - 1:
            r_ctx.append(g_cls_id[g[i+1]])
        else:
            r_ctx.append(g_cls_id['~'])
            
        window = l_ctx[::-1] + [g_cls_id[g[i]]] + r_ctx
        x.append(window)
        y.append(p_cls_id[p[i]])
        
    return (x, y)

for k in range(len(align_data)):
    xs, ys = get_xy2(align_data[k][0], align_data[k][1])
    for x, y in zip(xs, ys):
        XX.append(x)
        YY.append(y)

In [8]:
clf2 = RandomForestClassifier(n_estimators=10)
clf2 = clf2.fit(XX, YY)

In [9]:
g_from_cls = {}
p_from_cls = {}

for gr in g_cls_id:
    g_from_cls[g_cls_id[gr]] = gr
    
for ph in p_cls_id:
    p_from_cls[p_cls_id[ph]] = ph
    
print (g_cls_id['k'], g_from_cls[g_cls_id['k']])
print (phonemes[1], graphemes[1])

110 k
['N', 'AA', 'K', 'S', 'V', 'IH', 'L'] ['k', 'n', 'o', 'x', 'v', 'i', 'l', 'l', 'e']


In [10]:
def g2p(g):
    ch = letter_chunking(g.lower())
    g = []
    for c in ch:
        if len(c) > 1:
            g.append(c[0] + ':' + c[1])
        else:
            g.append(c[0])
    
    p = []
    for i in range(len(g)):
        l_ctx = []
        r_ctx = []
        j = i - 1
        while j >= 0 and len(l_ctx) < 4:
            l_ctx.append(g_cls_id.get(g[j], 0))
            l_ctx.append(p_cls_id[p[j]])
        while len(l_ctx) < 4:
            l_ctx.append(g_cls_id['~'])
            l_ctx.append(p_cls_id['_'])
            
        if i < len(g) - 1:
            r_ctx.append(g_cls_id.get(g[i+1], 0))
        else:
            r_ctx.append(g_cls_id['~'])
            
        window = l_ctx[::-1] + [g_cls_id.get(g[i], 0)] + r_ctx
        pp = p_from_cls[clf2.predict([window])[0]]
        p.append(pp)
        
    return p  

In [11]:
p_to_ch = {}

with open('phones.txt') as p_dct:
    lines = p_dct.readlines()
    for l in lines:
        ph, ch = l.split()
        p_to_ch[ph] = ch

In [12]:
def transform(ps):
    s = ""
    for ph in ps:
        if ':' in ph:
            a,b = ph.split(':')
            s += p_to_ch[a] + p_to_ch[b]
        elif ph == '_':
            continue
        else:
            s += p_to_ch[ph]
            
    return s

In [13]:
print(transform(g2p('knoxville')))

watCIqu


In [14]:
def test_g2p_model(g2p_model):
    test_words = []
    test_ids = []

    with open('test.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            test_ids.append(row[0])
            tw = ""
            for c in row[1].lower():
                if c.isalpha() or c == "'":
                    tw += c
            test_words.append(tw)
            
    with open('out.csv', 'w+') as out:
        out.write('Id,Transcription\n')
        for i in range(1, len(test_ids)):
            try:
                out.write(test_ids[i] + ',' + transform(g2p_model(test_words[i])) + '\n')
            except KeyError:
                print (test_words[i])

In [15]:
test_g2p_model(g2p)